In [1]:
import folium
import osmnx as ox
import pandas as pd
import os
import pickle
from collections import defaultdict
import cv2
import pylab as pl

from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# Manually-identified Cameras

In [2]:
cameras = pd.read_csv('data/manual_cameras_wboag.tsv', sep='\t')
cameras['Dome Camera'  ] = cameras['Dome Camera'  ].fillna(0)
cameras['Button Camera'] = cameras['Button Camera'].fillna(0)
cameras['Police Box'   ] = cameras['Police Box'   ].fillna(0)
cameras

,Latitude,Longitude,Name,Dome Camera,Button Camera,Police Box,Notes,Region,URL
0,42.365432,-71.061052,Causeway and Canal,1.0,0.0,1.0,NaN,West End,"https://www.google.com/maps/@42.3654325,-71.06..."
1,42.363682,-71.058375,New Chardon and John Fitzgerald,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3636822,-71.05..."
2,42.332230,-71.071605,Mass Ave and Gerard,0.0,3.0,1.0,NaN,Roxbury,"https://www.google.com/maps/@42.3322297,-71.07..."
3,42.330624,-71.070058,Mass Ave and Chesterton,0.0,1.0,1.0,NaN,Roxbury,"https://www.google.com/maps/@42.3306241,-71.07..."
4,42.326835,-71.066936,Mass Ave and Newmarket,1.0,0.0,0.0,NaN,Roxbury,"https://www.google.com/maps/@42.3268348,-71.06..."
...,...,...,...,...,...,...,...,...,...
148,42.364300,-71.063446,Staniford and Lomasney,2.0,1.0,1.0,NaN,West End,"https://www.google.com/maps/@42.3642999,-71.06..."
149,42.363862,-71.063149,Staniford and Merrimac,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3638619,-71.06..."
150,42.362982,-71.059962,New Chardon and Merrimac,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3629823,-71.05..."
151,42.352367,-71.073623,Berkeley,1.0,0.0,1.0,NaN,Back Bay,"https://www.google.com/maps/@42.3523674,-71.07..."


# Visualize Boston

In [3]:
# Create a map
b_folium = folium.Map(location=[42.3351457,-71.0748679], tiles='openstreetmap', zoom_start=13)

#add marker for each camera
colors = defaultdict(int)
for i,(_,row) in enumerate(cameras.iterrows()):
    num_cams = row['Dome Camera'] + row['Button Camera']
    
    if num_cams == 0:
        color = 'gray'
    elif num_cams == 1:
        color = 'red'
    elif num_cams == 2:
        color = 'orange'
    elif num_cams == 3:
        color = 'green'
    elif num_cams == 4:
        color = 'blue'
    else:
        color = 'purple'
    colors[color] += 1

    tooltip = ''
    popup = f'({row.Latitude},{row.Longitude})'
    #print(i, popup)
    icon = folium.map.Icon(color=color)
    coord = (row.Latitude, row.Longitude)
    folium.Marker(coord, popup=popup, tooltip=str(i), icon=icon).add_to(b_folium)

# display folium
print(colors)
b_folium

defaultdict(<class 'int'>, {'red': 106, 'green': 8, 'orange': 32, 'blue': 1, 'gray': 6})
